<a href="https://colab.research.google.com/github/iamscientist-1/LLM_Chatbot/blob/main/LLM_chatboat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# pip install PyPDF2

In [46]:
# pip install sentence-transformers

In [47]:
# !pip install faiss-gpu

In [48]:
# !pip install -q -U langchain transformers bitsandbytes accelerate

In [5]:
import PyPDF2
import os
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [27]:
def extract_text_from_pdf(file_path):
    page_data = []
    with open(file_path, 'rb') as file:

        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]

            text = page.extract_text()

            page_data.append(text)

    return page_data


file_path = '1908.02997.pdf'
pdf_text = extract_text_from_pdf(file_path)
for page_num, text in enumerate(pdf_text, start=1):
    print(f"Page {page_num}:\n{text}\n")


Page 1:
IEEE INTERNET OF THINGS JOURNAL, VOL. XX, NO. XX 1
Local Differential Privacy for Deep Learning
Pathum Chamikara Mahawaga Arachchige, Peter Bertok, Ibrahim Khalil, Dongxi Liu, Seyit Camtepe,
and Mohammed Atiquzzaman
The published article can be found at
https://doi.org/10.1109/JIOT.2019.2952146
Abstract —The internet of things (IoT) is transforming major
industries including but not limited to healthcare, agriculture,
ﬁnance, energy, and transportation. IoT platforms are contin-
ually improving with innovations such as the amalgamation
of software-deﬁned networks (SDN) and network function vir-
tualization (NFV) in the edge-cloud interplay. Deep learning
(DL) is becoming popular due to its remarkable accuracy when
trained with a massive amount of data, such as generated by
IoT. However, DL algorithms tend to leak privacy when trained
on highly sensitive crowd-sourced data such as medical data.
Existing privacy-preserving DL algorithms rely on the traditional
server-centric appr

In [28]:
# Pre-processing
def preprocess_page_data(page_data):

    processed_data = [re.sub(r'\s+', ' ', text) for text in page_data]
    return processed_data

page_data_prpocessed = [re.sub(r'\s+', ' ', text) for text in page_data]

In [8]:
# Initialize Sentence Transformers model
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
def build_embeddings_database(page_data_prpocessed, model):

    db = {}

    for page_num, text in enumerate(page_data_prpocessed):
        embeddings = model.encode(text)

        # Save embeddings and page number in the dictionary
        db[page_num] = {'text': text, 'embeddings': embeddings}

    return db


In [30]:
def extract_vectors_info(db):
    db_vectors = np.array([db[page_num]['embeddings'] for page_num in db.keys()])
    dimension = db_vectors.shape[1]    # dimensions of each vector
    n =  db_vectors.shape[0]   # number of vectors

    return db_vectors.shape, dimension, n


In [31]:
def create_and_train_index(db_vectors, dimension, nlist):
    quantiser = faiss.IndexFlatL2(dimension)
    index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

    print(index.is_trained)   # False
    index.train(db_vectors)  # train on the database vectors
    print(index.ntotal)   # 0
    index.add(db_vectors)   # add the vectors and update the index
    print(index.is_trained)  # True
    print(index.ntotal)   # Updated number of indexed vectors

    return index


In [32]:
def create_quantization_config():
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    return quantization_config

In [22]:
# model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [33]:

def create_model_with_quantization(model_name_or_path, quantization_config):
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        device_map="auto",
        quantization_config=quantization_config
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

    return model_4bit, tokenizer


In [34]:
def create_text_generation_pipeline(model, tokenizer, max_length=2500, do_sample=True, top_k=5, num_return_sequences=1):
    """
    Create a text generation pipeline with the specified model and tokenizer.
    model: The text generation model.
    tokenizer: The tokenizer associated with the model.
    max_length (int): The maximum length of the generated text.
    do_sample (bool): Whether or not to use sampling during generation.
    top_k (int): The number of highest probability tokens to sample from during generation.
    num_return_sequences (int): The number of sequences to generate.
    """
    pipeline_inst = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=max_length,
        do_sample=do_sample,
        top_k=top_k,
        num_return_sequences=num_return_sequences,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    return HuggingFacePipeline(pipeline=pipeline_inst)


In [24]:
##LLM
"""
Context = context
Prompt  = Prompt
User Query = User Query
"""


'\nContext = context\nPrompt  = Prompt\nUser Query = User Query\n'

In [43]:
def generate_response(user_query):
    def prepare_context(user_query):
        nprobe = 2  # find 2 most similar clusters
        n_query = 1
        k = 1  # return 3 nearest neighbours
        query_vectors = model.encode([user_query])
        distances, indices = index.search(query_vectors, k)
        context = ""
        for page_number in indices[0]:
            context += db[page_number]['text']
        return context

    def get_template(user_query, context):
        template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
        Context = {} \n\n
        Answer the question below from context below :
        {} [/INST] </s>
        """.format(context, user_query)
        return template

    context = prepare_context(user_query)
    template = get_template(user_query, context)
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"question": user_query})
    return response


In [44]:
user_query = "Tell me about Local Differential Privacy for Deep Learning"
print((generate_response(user_query)))



Local Differential Privacy (LDP) for Deep Learning is an extension of the standard differentially private mechanisms used to protect the privacy of individual data points in deep learning models. In LDP, the model training is done locally on individual devices, rather than on a centralized server, which provides additional privacy benefits by reducing the amount of sensitive data sent over the network. Additionally, LDP for deep learning allows for fine-tuning of the model on the local device, further improving its accuracy and performance.

LDP for deep learning is achieved by applying a random perturbation mechanism on the local data, such as Laplace mechanism or Gaussian mechanism, before sending it to the server for aggregation. The random perturbation adds noise to the data in a controlled manner, so that the model can still learn from the data while preserving the privacy of individual data points. The amount of noise added to the data is controlled by a privacy budget, which d